In [ ]:
!pip install fastapi uvicorn python-multipart pyngrok tensorflow tensorflow-hub opencv-python-headless


In [ ]:
# from fastapi import FastAPI, Form
# from fastapi.responses import FileResponse
# import uvicorn
# import os
# import requests
# import tensorflow as tf
# import tensorflow_hub as hub
# import numpy as np
# import cv2

# app = FastAPI()

# # Load the TF Hub style transfer model once
# print("🔄 Loading style transfer model...")
# hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
# print("✅ Model loaded.")

# def image_read(image_np):
#     max_dim = 512
#     image = tf.convert_to_tensor(image_np, dtype=tf.float32)
#     image = image / 255.0
#     shape = tf.cast(tf.shape(image)[:-1], tf.float32)
#     long_dim = max(shape)
#     scale = max_dim / long_dim
#     new_shape = tf.cast(shape * scale, tf.int32)
#     image = tf.image.resize(image, new_shape)
#     image = image[tf.newaxis, :]
#     return image

# def tensor_toimage(tensor):
#     tensor = tensor * 255
#     tensor = tf.cast(tensor, tf.uint8)
#     np_image = tensor.numpy()
#     if len(np_image.shape) > 3:
#         np_image = np_image[0]
#     return cv2.cvtColor(np_image, cv2.COLOR_RGB2BGR)

# @app.post("/stylize/")
# async def stylize_video(style_num: int = Form(...), video_url: str = Form(...)):
#     try:
#         artworks_dir = "/content/artworks"
#         style_path = os.path.join(artworks_dir, f"{style_num}.jpg")
#         input_path = "/content/input.mp4"
#         output_path = "/content/output_stylized.mp4"

#         if not os.path.exists(style_path):
#             return {"error": f"Style image {style_num}.jpg not found in artworks folder."}

#         # Download video from URL
#         print("📥 Downloading video...")
#         video = requests.get(video_url, stream=True)
#         with open(input_path, "wb") as f:
#             f.write(video.content)

#         print(f"🎨 Using style image: {style_path}")
#         style_image = cv2.imread(style_path)
#         style_image = cv2.cvtColor(style_image, cv2.COLOR_BGR2RGB)
#         style_image = image_read(style_image)

#         # Open video
#         cap = cv2.VideoCapture(input_path)
#         ret, first_frame = cap.read()
#         if not ret:
#             raise Exception("Couldn't read video.")
#         height, width, _ = first_frame.shape

#         out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 10, (width, height))

#         cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
#         frame_count = 0
#         while True:
#             ret, frame = cap.read()
#             if not ret:
#                 break
#             rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             content_image = image_read(rgb_frame)
#             stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
#             output_frame = tensor_toimage(stylized_image)
#             output_frame = cv2.resize(output_frame, (width, height))
#             out.write(output_frame)
#             frame_count += 1
#             if frame_count % 10 == 0:
#                 print(f"Processed {frame_count} frames...")
#         cap.release()
#         out.release()

#         print("🎞️ Style transfer complete.")
#         return FileResponse(output_path, media_type="video/mp4", filename="stylized_output.mp4")

#     except Exception as e:
#         print("❌ ERROR:", str(e))
#         return {"error": str(e)}


🔄 Loading style transfer model...
✅ Model loaded.


In [ ]:

!ngrok config add-authtoken 2z5UaLEwdbUaYWc9h0BLy0yFTvK_7cEf1SZdKa8kNgxLa1e8R  # Only once

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# import threading
# import uvicorn
# from pyngrok import ngrok

# # Start FastAPI server in background
# def run():
#     uvicorn.run(app, host="0.0.0.0", port=8000)


# thread = threading.Thread(target=run)
# thread.start()

# #  Start ngrok tunnel
# public_url = ngrok.connect(8000)
# print("🚀 Public URL:", public_url)


INFO:     Started server process [18026]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: Your account may not run more than 3 tunnels over a single ngrok agent session.\nThe tunnels already running on this session are:\ntn_2z5TL8B3EBEuhsZ8870IccWFngk, tn_2z5Tj53ovDw13sOQuS6MM9n1XVp, tn_2z5Tu2Hmo6BLoR9eaunYfloRgAb\n\r\n\r\nERR_NGROK_324\r\n"}}


In [ ]:
# !curl -X POST "https://1d96-34-80-52-198.ngrok-free.app/stylize/" -F style_num=16 -F video_url=https://res.cloudinary.com/dfmdmq24k/video/upload/v175016273/vcfgddwudavelvv1j3km.mp4 --output stylized_result.mp4


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0INFO:     34.80.52.198:0 - "POST /stylize/ HTTP/1.1" 200 OK
100   326  100    60  100   266     74    330 --:--:-- --:--:-- --:--:--   404


In [ ]:
# Install necessary packages
!pip install -q fastapi uvicorn python-multipart pyngrok tensorflow tensorflow-hub opencv-python-headless nest-asyncio

# Imports
import os
import cv2
import numpy as np
import requests
import threading
import tensorflow as tf
import tensorflow_hub as hub
import nest_asyncio
from fastapi import FastAPI, Form
from fastapi.responses import FileResponse
import uvicorn
from pyngrok import ngrok

# Enable nested event loop for Colab
nest_asyncio.apply()

# FastAPI app
app = FastAPI()

# Load style transfer model
print("🔄 Loading style transfer model...")
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
print("✅ Model loaded.")

# Helper: Prepare image for model
def image_read(image_np):
    max_dim = 512
    image = tf.convert_to_tensor(image_np, dtype=tf.float32)
    image = image / 255.0
    shape = tf.cast(tf.shape(image)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim
    new_shape = tf.cast(shape * scale, tf.int32)
    image = tf.image.resize(image, new_shape)
    image = image[tf.newaxis, :]
    return image

# Helper: Convert tensor to OpenCV image
def tensor_toimage(tensor):
    tensor = tensor * 255
    tensor = tf.cast(tensor, tf.uint8)
    np_image = tensor.numpy()
    if len(np_image.shape) > 3:
        np_image = np_image[0]
    return cv2.cvtColor(np_image, cv2.COLOR_RGB2BGR)

# Endpoint: Video style transfer
@app.post("/stylize/")
async def stylize_video(style_num: int = Form(...), video_url: str = Form(...)):
    try:
        artworks_dir = "/content/Artworks/"
        os.makedirs(artworks_dir, exist_ok=True)

        style_path = os.path.join(artworks_dir, f"{style_num}.jpg")
        input_path = "/content/input.mp4"
        output_path = "/content/output_stylized.mp4"
        print("📂 Looking for style at:", style_path)


        # Style image must exist
        if not os.path.exists(style_path):
            return {"error": f"Style image {style_num}.jpg not found in /content/Artworks."}

        # Download video
        print("📥 Downloading video...")
        video = requests.get(video_url, stream=True)
        with open(input_path, "wb") as f:
            f.write(video.content)

        # Load style image
        print(f"🎨 Using style image: {style_path}")
        style_image = cv2.imread(style_path)
        style_image = cv2.cvtColor(style_image, cv2.COLOR_BGR2RGB)
        style_image = image_read(style_image)
        print(f"📦 Downloaded size: {os.path.getsize(input_path)} bytes")


        # Process video
        cap = cv2.VideoCapture(input_path)
        ret, first_frame = cap.read()
        if not ret:
            raise Exception("Couldn't read video.")
        height, width, _ = first_frame.shape
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 10, (width, height))
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            content_image = image_read(rgb_frame)
            stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
            output_frame = tensor_toimage(stylized_image)
            output_frame = cv2.resize(output_frame, (width, height))
            out.write(output_frame)

            frame_count += 1
            if frame_count % 10 == 0:
                print(f"Processed {frame_count} frames...")

        cap.release()
        out.release()
        print("🎞️ Style transfer complete.")
        return FileResponse(output_path, media_type="video/mp4", filename="stylized_output.mp4")

    except Exception as e:
        print("❌ ERROR:", str(e))
        return {"error": str(e)}

# Auth token only needs to be added once
!ngrok config add-authtoken 2z5UaLEwdbUaYWc9h0BLy0yFTvK_7cEf1SZdKa8kNgxLa1e8R



🔄 Loading style transfer model...
✅ Model loaded.
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# !curl -X POST "https://9ab7-34-80-52-198.ngrok-free.app/stylize/" \
#   -F style_num=16 \
#   -F video_url=https://res.cloudinary.com/dahwwh212/video/upload/v1750508813/user1/dl1brgjfgxcvexslxdfu.mp4 \
#   --output stylized_result.mp4


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0📂 Looking for style at: /content/Artworks/16.jpg
📥 Downloading video...
100   336    0     0  100   336      0    267  0:00:01  0:00:01 --:--:--   267🎨 Using style image: /content/Artworks/16.jpg
📦 Downloaded size: 7424544 bytes
100   336    0     0  100   336      0      9  0:00:37  0:00:37 --:--:--     0Processed 10 frames...
100   336    0     0  100   336      0      4  0:01:24  0:01:09  0:00:15     0Processed 20 frames...
100   336    0     0  100   336      0      3  0:01:52  0:01:45  0:00:07     0Processed 30 frames...
100   336    0     0  100   336      0      2  0:02:48  0:02:17  0:00:31     0Processed 40 frames...
100   336    0     0  100   336      0      1  0:05:36  0:02:50  0:02:46     0Processed 50 frames...
100   336    0     0  100   33

In [ ]:
from threading import Thread
import time
from pyngrok import ngrok
import uvicorn

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print("🚀 Public URL:", public_url)

# Run FastAPI in background
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

Thread(target=run).start()

# Keep cell alive
while True:
    time.sleep(1)


INFO:     Started server process [35879]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 Public URL: NgrokTunnel: "https://7140-34-80-52-198.ngrok-free.app" -> "http://localhost:8000"


INFO:     2409:40f2:3044:13df:811f:1748:c51d:93a6:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2401:4900:1cb8:13b8:b14d:7ce2:a1d:5745:0 - "POST /stylize HTTP/1.1" 307 Temporary Redirect
📂 Looking for style at: /content/Artworks/16.jpg
📥 Downloading video...
🎨 Using style image: /content/Artworks/16.jpg
📦 Downloaded size: 4166423 bytes
Processed 10 frames...
Processed 20 frames...


KeyboardInterrupt: 